In [1]:
library(data.table)
library(stringi)
library(microbenchmark)

In [2]:
createTable <- function(rowCount) {
    gc()
    data.table(
    bucket = stri_rand_strings(rowCount, 2, pattern = "[a-z]"),
    qty = sample(1:100, rowCount, replace = TRUE),
    risk = sample(1:10, rowCount, replace = TRUE),
    weight = runif(rowCount, 0, 2)
    )
}

In [3]:
executeQuery <- function(t) {
    t[, .(
    NR = .N,
    TOTAL_QTY = sum(qty), AVG_QTY = mean(qty),
    TOTAL_RISK = sum(risk), AVG_RISK = mean(risk),
    WEIGHTED_QTY = weighted.mean(qty, weight),
    WEIGHTED_RISK = weighted.mean(risk, weight)
    ), by = bucket]
}

## 10 000

In [4]:
t <- createTable(10 * 1000)

In [5]:
summary(microbenchmark(executeQuery(t), times = 100))

expr,min,lq,mean,median,uq,max,neval
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
executeQuery(t),11.11668,11.46393,12.94802,12.29248,13.0872,21.2257,100


## 100 000

In [6]:
t <- createTable(100 * 1000)

In [7]:
summary(microbenchmark(executeQuery(t), times = 100))

expr,min,lq,mean,median,uq,max,neval
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
executeQuery(t),18.57509,19.22308,21.46852,19.55989,20.77382,34.30894,100


## 1 000 000

In [8]:
t <- createTable(1000 * 1000)

In [9]:
summary(microbenchmark(executeQuery(t), times = 100))

expr,min,lq,mean,median,uq,max,neval
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
executeQuery(t),108.4436,116.855,121.8517,118.9675,121.9411,192.9456,100


## 10 000 000

In [10]:
t <- createTable(10 * 1000 * 1000)

In [11]:
summary(microbenchmark(executeQuery(t), times = 100))

expr,min,lq,mean,median,uq,max,neval
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
executeQuery(t),1.039559,1.052319,1.084013,1.060406,1.084514,1.398073,100


# 100 000 000
We execute the tests ten times only!

In [12]:
t <- createTable(100 * 1000 * 1000)

In [13]:
summary(microbenchmark(executeQuery(t), times = 10))

expr,min,lq,mean,median,uq,max,neval
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
executeQuery(t),12.74955,12.89873,13.8897,13.39017,14.51712,16.57902,10


## 1 000 000 000
We execute the tests ten times only!

In [14]:
t <- createTable(1000 * 1000 * 1000)

ERROR: Error: cannot allocate vector of size 7.5 Gb


In [ ]:
summary(microbenchmark(executeQuery(t), times = 10))